In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
#converting categorical to numerical value
dataset=pd.get_dummies(dataset,dtype=int, drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [9]:
indep.shape

(400, 3)

In [10]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [11]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [12]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

# Define the parameter grid for GridSearchCV
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],  # Smoothing parameter
    'min_categories': [2, 3, 4]  # Minimum number of categories per feature
}

# Set up the GridSearchCV object
grid = GridSearchCV(CategoricalNB(), param_grid, 
                    refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

# Fitting the model for grid search
grid.fit(X_train, y_train)

# To see the best parameters and the best estimator
print("Best Parameters:", grid.best_params_)
print("Best Estimator:", grid.best_estimator_)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
Best Parameters: {'alpha': 0.1, 'min_categories': 2}
Best Estimator: CategoricalNB(alpha=0.1, min_categories=2)


C:\Anaconda\envs\aiml\Lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


In [13]:
# Store cross-validation results
cv_results = grid.cv_results_
# Uncomment to print all cross-validation results
# print(cv_results)

# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)


# Using the final best fit model to test X_test
grid_predictions = grid.predict(X_test)

In [14]:
# Compute and print the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,grid_predictions )  

In [15]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [16]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.84      0.91      0.87        85
           1       0.81      0.69      0.75        49

    accuracy                           0.83       134
   macro avg       0.82      0.80      0.81       134
weighted avg       0.83      0.83      0.83       134



In [17]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

np.float64(0.870828331332533)

In [18]:
print(cm)

[[77  8]
 [15 34]]


In [19]:
grid.predict([[40,300,1,]])

C:\Anaconda\envs\aiml\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(


array([0])

In [20]:
table=pd.DataFrame.from_dict(re)

In [21]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_min_categories,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.047029,0.008085,0.014339,0.003762,0.1,2,"{'alpha': 0.1, 'min_categories': 2}",0.735616,0.743112,0.710831,0.805463,NaN,NaN,NaN,1
1,0.036735,0.005430,0.014280,0.004971,0.1,3,"{'alpha': 0.1, 'min_categories': 3}",0.735616,0.743112,0.710831,0.805463,NaN,NaN,NaN,1
2,0.034805,0.002952,0.012763,0.005134,0.1,4,"{'alpha': 0.1, 'min_categories': 4}",0.735616,0.743112,0.710831,0.805463,NaN,NaN,NaN,1
3,0.030415,0.002653,0.019221,0.006408,0.5,2,"{'alpha': 0.5, 'min_categories': 2}",0.751984,0.782633,0.688741,0.783837,NaN,NaN,NaN,1
4,0.042026,0.006402,0.010255,0.004357,0.5,3,"{'alpha': 0.5, 'min_categories': 3}",0.751984,0.782633,0.688741,0.783837,NaN,NaN,NaN,1
5,0.044748,0.007043,0.018499,0.007496,0.5,4,"{'alpha': 0.5, 'min_categories': 4}",0.751984,0.782633,0.688741,0.783837,NaN,NaN,NaN,1
6,0.045215,0.009850,0.014323,0.003023,1.0,2,"{'alpha': 1.0, 'min_categories': 2}",0.726566,0.776838,0.673750,0.818941,NaN,NaN,NaN,1
7,0.045469,0.005316,0.012624,0.006143,1.0,3,"{'alpha': 1.0, 'min_categories': 3}",0.726566,0.776838,0.663557,0.818941,NaN,NaN,NaN,1
8,0.052663,0.017223,0.015082,0.002145,1.0,4,"{'alpha': 1.0, 'min_categories': 4}",0.726566,0.776838,0.663557,0.818941,NaN,NaN,NaN,1
9,0.042190,0.006293,0.014434,0.003460,1.5,2,"{'alpha': 1.5, 'min_categories': 2}",0.699882,0.776838,0.663557,0.796094,NaN,NaN,NaN,1
